# Package Setup

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv3D, BatchNormalization, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model


KeyboardInterrupt



In [ ]:
# Constants
IMG_SIZE = 256
FRAME_COUNT = 10

# Frames and Dataset Pre-processing

In [ ]:
# Function to preprocess individual frames
def preprocess_frame(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))  # Resize to IMG_SIZE x IMG_SIZE
    frame = frame.astype('float32') / 255.0  # Normalize pixel values
    return frame

# Function to extract and preprocess frames from a video
def extract_frames_from_video(video_path, frame_count=FRAME_COUNT):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = preprocess_frame(frame)
        frames.append(frame)

        # Stop once we have enough frames
        if len(frames) == frame_count:
            break

    cap.release()
    
    # If fewer than frame_count frames, pad with the last frame
    while len(frames) < frame_count:
        frames.append(frames[-1])
    
    return np.array(frames)


# Loading Dataset

In [ ]:
# Paths to real and fake video datasets
real_videos_dir = '/kaggle/input/celeb-df-v2/Celeb-real'
fake_videos_dir = '/kaggle/input/celeb-df-v2/Celeb-synthesis'

In [ ]:
# Function to load dataset
def load_dataset(data_dir, label, frame_count=FRAME_COUNT):
    data = []
    labels = []
    
    for video_file in os.listdir(data_dir):
        video_path = os.path.join(data_dir, video_file)
        frames = extract_frames_from_video(video_path, frame_count)
        data.append(frames)
        labels.append(label)
    
    return np.array(data), np.array(labels)

# Load real and fake video datasets
real_videos, real_labels = load_dataset(real_videos_dir, label=0)
fake_videos, fake_labels = load_dataset(fake_videos_dir, label=1)


# Combine and Split Dataset

In [ ]:
# Combine real and fake datasets
videos = np.concatenate([real_videos, fake_videos], axis=0)
labels = np.concatenate([real_labels, fake_labels], axis=0)

# Split into training, validation, and testing sets
train_videos, test_videos, train_labels, test_labels = train_test_split(videos, labels, test_size=0.2, random_state=42)
train_videos, val_videos, train_labels, val_labels = train_test_split(train_videos, train_labels, test_size=0.2, random_state=42)

# Create TensorFlow datasets for batch processing
train_data = tf.data.Dataset.from_tensor_slices((train_videos, train_labels)).batch(16)
validation_data = tf.data.Dataset.from_tensor_slices((val_videos, val_labels)).batch(16)
test_data = tf.data.Dataset.from_tensor_slices((test_videos, test_labels)).batch(16)